Using the functions and loops created in the webscraper notebook.  Here I'm going to loop through one webpage and get all 20 tables into one dataframe

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, NoSuchWindowException
from selectorlib import Extractor
import bs4
import requests
import json
import time
import urllib.request
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import csv
import pandas as pd
import numpy as np
from datetime import date
import py_files.transfer_webscraper as transfer_webscraper

# This is how you call it:
# transfer_webscraper.transfer_value_gather(table)

In [3]:
ChromeOptions = webdriver.ChromeOptions()
ChromeOptions.add_argument('ignore-certificate-errors')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=ChromeOptions)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/ericroberts/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


In [4]:
leagues = ['premier-league','ligue-1','primera-division','1-bundesliga','serie-a','primeira-liga','eredivisie','premier-liga']
league_abbreviation = ['GB1','FR1','ES1','L1','IT1','PO1','NL1','RU1']
years = ['2018','2019','2020']
transfer_window = ['s_w=w','s_w=s'] #winter and summer

Top_8_leagues_past_6_windows = pd.DataFrame()
for j in range(len(leagues)):
    all_years = pd.DataFrame()
    for year in years:
        both_transfer_windows = pd.DataFrame()
        for window in transfer_window:
            
            #This creates the appropriate url for each league and season and window
            #List were created from transfermarkt urls (see below)
            url = f'https://www.transfermarkt.co.uk/{leagues[j]}/sommertransfers/wettbewerb/{league_abbreviation[j]}/plus/?saison_id={year}&{window}&leihe=1&intern=0&intern=1'
            driver.get(url)
            time.sleep(np.random.randint(1,3))
            html = driver.execute_script('return document.body.innerHTML;')
            soup = bs4.BeautifulSoup(html,'lxml')
            
            #Creates a list of tables of all incoming and outgoing players for clubs
            #All the transfer information are stored under the tag class_="responsive-table"
            #The information is stored as 2 tables for each club 1 for in's and 1 for out's 
            #at each club - some windows have fewer than 40 observations, e.g., winter windows with no activity)
            #
            #this line returns a list of length 40.  The function below extracts the appropriate information.
            response_table = soup.find_all(class_="responsive-table")
            
            #Reduces that list to only puchases by that club
            list_of_table_of_purchases = []
            response_table = soup.find_all(class_="responsive-table")
            for i in range(len(response_table)):
                if i % 2 == 0:
                    list_of_table_of_purchases.append(response_table[i])
            
            #Creates list of clubs to create the buying_club column
            #The transfer info tables are labled with the team name but under a different tag.
            #This creates a list of the clubs in the league in that season; mirrors the order that 
            #response_table is ordered.  Use this along with above function when looking through response_tables to add
            #the purchasing club as a column to the df.
            list_of_clubs = []
            table_header = soup.find_all(class_='table-header')
            i=2
            while i < (len(table_header)-1):
                club = table_header[i].find_all(class_="vereinprofil_tooltip tooltipstered")
                list_of_clubs.append(club[1].text)
                i += 1
            list_of_clubs

            #Looping over the list tables (list_of_table_of_purchases)
            #scraping using scraping function transfer_webscraper.py
            #adds the buying club as a column
            one_transfer_window = pd.DataFrame()
            for n in range(len(list_of_clubs)):
                df = transfer_webscraper.transfer_value_gather(list_of_table_of_purchases[n])
                df['buying_club'] = list_of_clubs[n]
                #the function itself returns only a df for one team in one league in one window in one year
                #this loop goes over all the teams in that league in that window in that year
                one_transfer_window = one_transfer_window.append(df)
                
            #adds the window as a column        
            one_transfer_window['window'] = window
            both_transfer_windows = both_transfer_windows.append(one_transfer_window)
        #adds the year as a column    
        both_transfer_windows['year'] = year
        all_years = all_years.append(both_transfer_windows)
    #adds the destination league as a column    
    all_years['buying_league'] = league_abbreviation[j]
    Top_8_leagues_past_6_windows = Top_8_leagues_past_6_windows.append(all_years)

In [5]:
Top_8_leagues_past_6_windows.shape

(9666, 12)

In [6]:
Top_8_leagues_past_6_windows.head()

,player,age,nationality,position,selling_club,previous_league,est_market_value,fee,buying_club,window,year,buying_league
0,Ante Palaversa,18,Croatia,Defensive Midfield,HNK Hajduk Split,Croatia,£495Th.,£5.67m,Manchester City,s_w=w,2018,GB1
1,Ko Itakura,21,Japan,Centre-Back,Kawasaki Frontale,Japan,£630Th.,£990Th.,Manchester City,s_w=w,2018,GB1
2,Yangel Herrera,20,Venezuela,Central Midfield,New York City FC,United States,£900Th.,"End of loanDec 31, 2018",Manchester City,s_w=w,2018,GB1
3,Marlos Moreno,22,Colombia,Right Winger,Clube de Regatas do Flamengo,Brazil,£450Th.,"End of loanDec 31, 2018",Manchester City,s_w=w,2018,GB1
4,Anthony Cáceres,26,Australia,Central Midfield,Melbourne City FC,Australia,£450Th.,"End of loanDec 31, 2018",Manchester City,s_w=w,2018,GB1


In [7]:
Top_8_leagues_past_6_windows.tail()

,player,age,nationality,position,selling_club,previous_league,est_market_value,fee,buying_club,window,year,buying_league
9,Arseni Logashov,29,Russia,Right-Back,FK Rostov,Russia,£630Th.,loan transfer,FK Khimki,s_w=s,2020,RU1
10,Aleksandr Dolgov,22,Russia,Centre-Forward,FK Rostov,Russia,£360Th.,loan transfer,FK Khimki,s_w=s,2020,RU1
11,Ilya Kamyshev,23,Russia,Defensive Midfield,Chertanovo Moscow,Russia,£225Th.,loan transfer,FK Khimki,s_w=s,2020,RU1
12,Aleksandr Filin,24,Russia,Centre-Back,PFK Tambov,Russia,£810Th.,"End of loanJul 31, 2020",FK Khimki,s_w=s,2020,RU1
13,Andrey Mendel,25,Russia,Defensive Midfield,Volgar Astrakhan,Russia,£315Th.,"End of loanJun 30, 2020",FK Khimki,s_w=s,2020,RU1


In [8]:
Top_8_leagues_past_6_windows.to_csv('data/Top_8_leagues_past_6_windows.csv')

In [ ]:
# From the transfermarkt website creating these lists for the url's 
# and to add variables:

##These are the top 8 leagues in the world
#  https://www.transfermarkt.co.uk/premier-league/sommertransfers/wettbewerb/GB1/saison_id/2020
#  https://www.transfermarkt.co.uk/ligue-1/sommertransfers/wettbewerb/FR1
#  https://www.transfermarkt.co.uk/primera-division/sommertransfers/wettbewerb/ES1
#  https://www.transfermarkt.co.uk/1-bundesliga/sommertransfers/wettbewerb/L1
#  https://www.transfermarkt.co.uk/serie-a/sommertransfers/wettbewerb/IT1
#  https://www.transfermarkt.co.uk/primeira-liga/sommertransfers/wettbewerb/PO1
#  https://www.transfermarkt.co.uk/premier-liga/sommertransfers/wettbewerb/RU1
#  https://www.transfermarkt.co.uk/eredivisie/sommertransfers/wettbewerb/NL1
    
##Use these to compare year and the winter and summer window:    
#  https://www.transfermarkt.co.uk/premier-league/sommertransfers/wettbewerb/GB1/plus/?saison_id=2019&s_w=w&leihe=1&intern=0&intern=1
#  https://www.transfermarkt.co.uk/premier-league/sommertransfers/wettbewerb/GB1/plus/?saison_id=2020&s_w=s&leihe=1&intern=0&intern=1    